In [1]:
%pip install quantstats
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [2]:
import quantstats as qs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.linear_model import LinearRegression

In [48]:
# Change day and round here
path1 = f"./data/day_0/prices_round_1_day_-2.csv"
path2 = f"./data/day_0/prices_round_1_day_-1.csv"
path3 = f"./data/day_0/prices_round_1_day_0.csv"

In [49]:
# Get data
data1 = pd.read_csv(path1, delimiter=";", parse_dates=True)
data2 = pd.read_csv(path2, delimiter=";", parse_dates=True)
data3 = pd.read_csv(path3, delimiter=";", parse_dates=True)

In [52]:
parsed_data_1 = data1[['day', 'timestamp', 'product', 'mid_price']].copy()
parsed_data_2 = data2[['day', 'timestamp', 'product', 'mid_price']].copy()
parsed_data_3 = data3[['day', 'timestamp', 'product', 'mid_price']].copy()

In [56]:
parsed_data = pd.concat([parsed_data_1, parsed_data_2, parsed_data_3], ignore_index=True)
parsed_data.drop(parsed_data[parsed_data["product"] == "AMETHYSTS"].index, inplace=True)
parsed_data

,day,timestamp,product,mid_price
1,-2,0,STARFRUIT,5000.0
3,-2,100,STARFRUIT,5002.0
4,-2,200,STARFRUIT,5000.0
7,-2,300,STARFRUIT,4999.5
9,-2,400,STARFRUIT,4998.5
...,...,...,...,...
59990,0,999500,STARFRUIT,5052.5
59992,0,999600,STARFRUIT,5051.0
59995,0,999700,STARFRUIT,5053.5
59996,0,999800,STARFRUIT,5049.5


In [57]:
def calculate_price_index(p1, p2, p3, type):
    l = [p1, p2, p3]
    
    for i in range(len(l)):
        if not isinstance(l[i], int):
            del l[i]

    if type == 'bid':
        return l.index(max(l))
    else:
        return l.index(min(l))

In [58]:
lags = 4
for i in range(1, lags + 1):
    parsed_data[f'lag{i}'] = parsed_data['mid_price'].shift(i)

parsed_data.dropna(inplace=True)
parsed_data

,day,timestamp,product,mid_price,lag1,lag2,lag3,lag4
9,-2,400,STARFRUIT,4998.5,4999.5,5000.0,5002.0,5000.0
11,-2,500,STARFRUIT,5002.0,4998.5,4999.5,5000.0,5002.0
12,-2,600,STARFRUIT,4999.5,5002.0,4998.5,4999.5,5000.0
15,-2,700,STARFRUIT,5000.0,4999.5,5002.0,4998.5,4999.5
17,-2,800,STARFRUIT,5002.5,5000.0,4999.5,5002.0,4998.5
...,...,...,...,...,...,...,...,...
59990,0,999500,STARFRUIT,5052.5,5051.5,5052.0,5051.5,5052.0
59992,0,999600,STARFRUIT,5051.0,5052.5,5051.5,5052.0,5051.5
59995,0,999700,STARFRUIT,5053.5,5051.0,5052.5,5051.5,5052.0
59996,0,999800,STARFRUIT,5049.5,5053.5,5051.0,5052.5,5051.5


In [59]:
x = parsed_data[['lag1', 'lag2', 'lag3', 'lag4']]
x

,lag1,lag2,lag3,lag4
9,4999.5,5000.0,5002.0,5000.0
11,4998.5,4999.5,5000.0,5002.0
12,5002.0,4998.5,4999.5,5000.0
15,4999.5,5002.0,4998.5,4999.5
17,5000.0,4999.5,5002.0,4998.5
...,...,...,...,...
59990,5051.5,5052.0,5051.5,5052.0
59992,5052.5,5051.5,5052.0,5051.5
59995,5051.0,5052.5,5051.5,5052.0
59996,5053.5,5051.0,5052.5,5051.5


In [60]:
y = parsed_data['mid_price']
y

9        4998.5
11       5002.0
12       4999.5
15       5000.0
17       5002.5
          ...  
59990    5052.5
59992    5051.0
59995    5053.5
59996    5049.5
59998    5051.0
Name: mid_price, Length: 29996, dtype: float64

In [61]:
model = LinearRegression()
model.fit(x, y)
pred = model.predict(x)
parsed_data["pred_price"] = pred
parsed_data

,day,timestamp,product,mid_price,lag1,lag2,lag3,lag4,pred_price
9,-2,400,STARFRUIT,4998.5,4999.5,5000.0,5002.0,5000.0,5000.265815
11,-2,500,STARFRUIT,5002.0,4998.5,4999.5,5000.0,5002.0,4999.756075
12,-2,600,STARFRUIT,4999.5,5002.0,4998.5,4999.5,5000.0,5000.209366
15,-2,700,STARFRUIT,5000.0,4999.5,5002.0,4998.5,4999.5,4999.966485
17,-2,800,STARFRUIT,5002.5,5000.0,4999.5,5002.0,4998.5,5000.022682
...,...,...,...,...,...,...,...,...,...
59990,0,999500,STARFRUIT,5052.5,5051.5,5052.0,5051.5,5052.0,5051.722262
59992,0,999600,STARFRUIT,5051.0,5052.5,5051.5,5052.0,5051.5,5051.942855
59995,0,999700,STARFRUIT,5053.5,5051.0,5052.5,5051.5,5052.0,5051.681912
59996,0,999800,STARFRUIT,5049.5,5053.5,5051.0,5052.5,5051.5,5052.257943


In [62]:
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)
print("Accuracy:", model.score(x, y))

Coefficients: [0.34176867 0.26106908 0.20770677 0.18898843]
Intercept: 2.3564943532555844
Accuracy: 0.9981002615959415
